In [234]:
from collections import OrderedDict
from typing import List

import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from robotpy_apriltag import AprilTagDetector, AprilTagPoseEstimator, AprilTagDetection
from scipy.spatial.transform import Rotation
from wpimath.geometry import Transform3d


In [2]:
matplotlib.use('TkAgg')

In [3]:
paths = [
    ".\\data\\apriltags_test\\best_case.png",
    ".\\data\\apriltags_test\\higher.png",
    ".\\data\\apriltags_test\\highest.tif",
    "data/apriltags_run2/0001SET/000/IMG_0031_2.tif"
]
paths_cross_ref = [
    ".\\data\\apriltags_test\\highest_cross_ref\\IMG_0259_1.tif",
    ".\\data\\apriltags_test\\highest_cross_ref\\IMG_0259_2.tif",
    ".\\data\\apriltags_test\\highest_cross_ref\\IMG_0259_3.tif",
    ".\\data\\apriltags_test\\highest_cross_ref\\IMG_0259_4.tif",
    ".\\data\\apriltags_test\\highest_cross_ref\\IMG_0259_5.tif",
]
images = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in paths]
images_cross_ref = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in paths_cross_ref]

In [6]:
families = [
    "tag16h5",
    "tag25h9",
    "tagStandard41h12",
    "tagStandard52h13",
    "tagCircle49h12",
    "tagCircle21h7",
]
detectors: List[AprilTagDetector] = []
for family in families:
    d = AprilTagDetector()
    d.addFamily(family)
    config = AprilTagDetector.Config()
    config.quadDecimate = 1.0
    config.numThreads = 4
    config.refineEdges = 1.0
    d.setConfig(config)
    detectors.append(d)

cmap = plt.get_cmap('viridis')
colors = cmap(np.linspace(0, 1, len(detectors)))

In [7]:
def verify_detections(tag, valid_ids=None) -> bool:
    if valid_ids is None:
        valid_ids = [0, 4, 9]
    return tag.getId() in valid_ids

In [92]:
def detect_tags(img, show_id=False):
    plt.imshow(img, cmap="gray")
    plt.axis("off")
    for detector, color in zip(detectors, colors):
        tags: List[AprilTagDetection] = detector.detect(img)
        for tag in tags:
            if not verify_detections(tag):
                continue
            plt.scatter(tag.getCenter().x, tag.getCenter().y, marker="o", color=color, s=20, label=tag.getFamily())
            if show_id:
                plt.text(tag.getCenter().x + 50, tag.getCenter().y - 50, tag.getId(), color="black", ha="center",
                         va="center",
                         bbox=dict(boxstyle="round",
                                   ec=(1., 0.5, 0.5),
                                   fc=(1., 0.8, 0.8),
                                   ))
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()

In [93]:
detect_tags(images[3], True)

TypeError: Axes3D.text() missing 1 required positional argument: 's'

In [8]:
for i, img in enumerate(images):
    detect_tags(img, True)

## Cross Referencing between Bands

In [8]:
def detect_tags_cross_reference(imgs):
    plt.imshow(imgs[1], cmap="gray")
    plt.axis("off")
    for img in imgs:
        for detector, color in zip(detectors, colors):
            tags: List[AprilTagDetection] = detector.detect(img)
            for tag in tags:
                plt.scatter(tag.getCenter().x, tag.getCenter().y, marker="o", color=color, s=10, label=tag.getFamily())
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()

In [9]:
detect_tags_cross_reference(images_cross_ref)

## 3D Pose Estimation

In [238]:
# Hyperparameters
horizontal_focal_length_pixels = 1581.7867974691412
horizontal_focal_center_pixels = 678.6724626822399
vertical_focal_length_pixels = 1581.7867974691412
vertical_focal_center_pixels = 529.4318832108801
tag_size = 0.5
panel_size = 1.0

In [237]:
# Helper functions

def rotate_vector(vector, axis, degrees):
    theta = degrees * (np.pi / 180.0)
    axis = axis / norm(axis)  # normalize the rotation vector first
    rot = Rotation.from_rotvec(theta * axis)
    new_v = rot.apply(vector)
    return new_v


def project_to_image_plane(X, Y, Z,
                           horizontal_focal_length_pixels, vertical_focal_length_pixels,
                           horizontal_focal_center_pixels, vertical_focal_center_pixels):
    """
    Projects a 3D world point (X, Y, Z) into 2D image plane coordinates (u, v).

    Parameters:
    X, Y, Z: float
        3D coordinates of the point in world space.
    horizontal_focal_length_pixels: float
        Focal length of the camera in the horizontal direction (in pixels).
    vertical_focal_length_pixels: float
        Focal length of the camera in the vertical direction (in pixels).
    horizontal_focal_center_pixels: float
        The x-coordinate of the principal point (optical center) in the image.
    vertical_focal_center_pixels: float
        The y-coordinate of the principal point (optical center) in the image.

    Returns:
    u, v: float
        2D pixel coordinates in the image plane.
    """
    # Project onto the 2D image plane
    u = (X * horizontal_focal_length_pixels) / Z + horizontal_focal_center_pixels
    v = (Y * vertical_focal_length_pixels) / Z + vertical_focal_center_pixels

    return u, v

In [244]:
def pose_estimate_tags_and_mark_panels(img, valid_ids=None):
    fig_3d = plt.figure()
    ax_3d = fig_3d.add_subplot(111, projection='3d')
    ax_3d.set_xlim(-5, 5)
    ax_3d.set_ylim(-5, 5)
    ax_3d.set_zlim(25)
    color = "green"

    def print_location(point):
        ax_3d.scatter(*point, marker="o", color=color, s=2)

    # Print image
    R = np.eye(3)
    t = np.zeros((1, 3))
    img_color = cv2.cvtColor(img, cv2.COLOR_RGB2BGRA)
    #plotImage(ax_3d, img_color, R, t, size=[1000,1000], img_scale=16)

    # Print Camera location
    print_location([0, 0, 0])
    pose_estimator = AprilTagPoseEstimator(
        AprilTagPoseEstimator.Config(tag_size, horizontal_focal_length_pixels,
                                     vertical_focal_length_pixels,
                                     horizontal_focal_center_pixels, vertical_focal_center_pixels))
    for detector, color in zip(detectors, colors):
        tags: List[AprilTagDetection] = detector.detect(img)
        for tag in tags:
            if not verify_detections(tag, valid_ids):
                continue
            estimate: Transform3d = pose_estimator.estimate(tag)
            # Plot the 3D pose estimate
            rotation = [estimate.rotation().x, estimate.rotation().y, estimate.rotation().z]
            rotation = rotation / np.linalg.norm(rotation)
            rotation = rotation * tag_size
            #ax_3d.quiver(origin[0], origin[1], origin[2], *rotation)
            tag_up_dir = rotate_vector(rotation, [0, 1, 0], 90.0)
            border_between_tag_and_panel = estimate.translation() + tag_up_dir
            edgeA = border_between_tag_and_panel + rotate_vector(tag_up_dir * (1 / tag_size) * (panel_size / 2),
                                                                 [0, 0, 1],
                                                                 90.0)
            edgeB = border_between_tag_and_panel - rotate_vector(tag_up_dir * (1 / tag_size) * (panel_size / 2),
                                                                 [0, 0, 1],
                                                                 90.0)
            edgeC = border_between_tag_and_panel + tag_up_dir * (1 / tag_size) * panel_size - rotate_vector(
                tag_up_dir * (1 / tag_size) * (panel_size / 2), [0, 0, 1],
                90.0)
            edgeD = border_between_tag_and_panel + tag_up_dir * (1 / tag_size) * panel_size + rotate_vector(
                tag_up_dir * (1 / tag_size) * (panel_size / 2), [0, 0, 1],
                90.0)

            ax_3d.quiver(*estimate.translation().toVector(), *tag_up_dir, color=color)
            print_location(border_between_tag_and_panel )
            print_location(estimate.translation())
            print_location(edgeA)
            print_location(edgeB)
            print_location(edgeC)
            print_location(edgeD)
            ax_3d.quiver(*edgeA, *(edgeB - edgeA), color=color)
            ax_3d.quiver(*edgeB, *(edgeC - edgeB), color=color)
            ax_3d.quiver(*edgeC, *(edgeD - edgeC), color=color)
            ax_3d.quiver(*edgeD, *(edgeA - edgeD), color=color)
    fig_3d.show()

In [245]:
pose_estimate_tags_and_mark_panels(images[3], [4])

Projected pixel coordinates: (u, v) = (995.0298221760681, 1162.1466021985366)


In [251]:
def get_panel_edges(img, valid_ids):
    panels = []
    for detector, family in zip(detectors, families):
        tags: List[AprilTagDetection] = detector.detect(img)
        tag_size_adjusted = tag_size
        if family == "tag25h9":
            tag_size_adjusted = 0.382
        elif family == "tagCircle21h7":
            tag_size_adjusted = 0.47
        pose_estimator = AprilTagPoseEstimator(
            AprilTagPoseEstimator.Config(tag_size_adjusted, horizontal_focal_length_pixels,
                                         vertical_focal_length_pixels,
                                         horizontal_focal_center_pixels, vertical_focal_center_pixels))
        for tag in tags:
            if not verify_detections(tag, valid_ids):
                continue
            estimate: Transform3d = pose_estimator.estimate(tag)
            # The estimated rotation is a vector that points up and away form the panel (Z axis is up towards the camera)
            tag_orientation = [estimate.rotation().x, estimate.rotation().y, estimate.rotation().z]
            # Scale the rotation vector to the tag_size
            tag_orientation = tag_orientation / np.linalg.norm(tag_orientation)
            tag_orientation = tag_orientation * tag_size_adjusted
            # The tag orientation is then rotated to face the "up" side of the tag when thinking in terms of a 2d paper print. (Towards the panel on the ground)
            tag_up_dir = rotate_vector(tag_orientation, [0, 1, 0], 90.0)
            # Calculate vectors that point in the direction of the panel edges
            tag_panel_border = estimate.translation() + tag_up_dir
            panel_length = tag_up_dir * (1 / tag_size_adjusted) * panel_size
            half_panel_length = panel_length / 2
            panel_midpoint_to_edge = rotate_vector(half_panel_length, [0, 0, 1], 90.0)
            edgeA = tag_panel_border + panel_midpoint_to_edge
            edgeB = tag_panel_border - panel_midpoint_to_edge
            edgeC = tag_panel_border + panel_length - panel_midpoint_to_edge
            edgeD = tag_panel_border + panel_length + panel_midpoint_to_edge
            panels.append((estimate, (edgeA, edgeB, edgeC, edgeD)))
    return panels


def project_edges(edges):
    return [project_to_image_plane(*edge,
                                   horizontal_focal_length_pixels,
                                   vertical_focal_length_pixels,
                                   horizontal_focal_center_pixels,
                                   vertical_focal_center_pixels) for edge in edges]

In [268]:
def show_panels_in_image_by_pose_estimation(img, valid_ids):
    fig_2d = plt.figure()
    ax = fig_2d.subplots(1, 1)
    ax.imshow(img, cmap="grey")
    panels_3d = get_panel_edges(img, valid_ids)
    panels = [(project_edges([estimate.translation()]), project_edges(edges)) for (estimate, edges) in panels_3d]
    for ([estimate], edges), color in zip(panels, colors):
        ax.scatter(estimate[0], estimate[1], color=color)
        x, y = zip(*edges)

        # Append the first point to the end to close the rectangle/polygon
        x = list(x) + [x[0]]
        y = list(y) + [y[0]]
        ax.plot(x, y, color=color)
    fig_2d.show()

In [269]:
show_panels_in_image_by_pose_estimation(images[3], [4])